In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,scale,QuantileTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski']

In [18]:
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

df_train = df.drop("target", axis=1)

X = df_train.values
#X_test = df_test.values

# print(X.shape)
# print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)

unique_labels = np.unique(y)

tf = TfidfTransformer()

print("Construction des centroïdes")

X_tf = tf.fit_transform(X)

centroids = []
for label in unique_labels:
    same_labels = X_tf[np.where(y == label)]
    centroids.append(same_labels.sum(axis = 0))
centroids = np.asarray(centroids).reshape(9,93)
print(centroids.shape)
print("Construction terminée")
X_dc = pairwise_distances(X_tf, centroids, metric='euclidean')
#X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

X_train, X_test, y_train, y_test = train_test_split(X_dc, y, test_size=0.2, random_state=42)

print("X_train shape : ", X_train.shape)
print("X_test shape : ", X_test.shape)


<class 'numpy.ndarray'>
(61878,)
Construction des centroïdes
(9, 93)
Construction terminée
X_train shape :  (49502, 9)
X_test shape :  (12376, 9)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
%time clf.fit(X_train, y_train)
pred_prob = clf.predict_proba(X_test)
pred = clf.predict(X_test)

print("Kaggle log loss :", log_loss(y_test, pred_prob))
print(classification_report(y_test, pred))

In [ ]:
print(X_train.shape)

In [19]:
clf = KNeighborsClassifier(n_neighbors=10, n_jobs=6)
%time clf.fit(X_train, y_train)
%time pred_prob = clf.predict_proba(X_test)
pred = clf.predict(X_test)

print("Kaggle log loss :", log_loss(y_test, pred_prob))
print(classification_report(y_test, pred))

CPU times: user 93 ms, sys: 2.85 ms, total: 95.9 ms
Wall time: 95.1 ms
CPU times: user 1.79 s, sys: 26.9 ms, total: 1.81 s
Wall time: 644 ms
Kaggle log loss : 1.89225736074
             precision    recall  f1-score   support

          0       0.55      0.32      0.40       399
          1       0.66      0.84      0.74      3178
          2       0.49      0.37      0.42      1561
          3       0.54      0.15      0.24       538
          4       0.96      0.96      0.96       565
          5       0.91      0.92      0.91      2884
          6       0.70      0.59      0.64       552
          7       0.83      0.89      0.86      1674
          8       0.81      0.81      0.81      1025

avg / total       0.74      0.75      0.74     12376



In [20]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc))



CPU times: user 16.9 s, sys: 80.5 ms, total: 17 s
Wall time: 4.65 s
CPU times: user 248 ms, sys: 25.3 ms, total: 273 ms
Wall time: 109 ms
Kaggle log loss metric : 1.71
Accuracy : 0.33
             precision    recall  f1-score   support

          0       0.64      0.09      0.15       399
          1       0.28      0.99      0.43      3178
          2       0.67      0.02      0.04      1561
          3       0.56      0.01      0.02       538
          4       0.96      0.97      0.96       565
          5       0.70      0.02      0.05      2884
          6       0.86      0.15      0.26       552
          7       0.93      0.04      0.08      1674
          8       0.90      0.06      0.12      1025

avg / total       0.65      0.33      0.21     12376



In [ ]:
clf = SVC(verbose=True, probability=True)
%time clf.fit(X_train, y_train)
pred_prob = clf.predict_proba(X_test)
pred = clf.predict(X_test)

print("Kaggle log loss :", log_loss(y_test, pred_prob))
print(classification_report(y_test, pred))